# RFM-анализ 

В анализе важно изучить клиентов имеющие бонусные карты, так как по результатам анализа необходимо отследить поведение клиентов, которые уже завели карты, а так же сделать рассылку по результатам rfm - сегментов


Датасет содержит колонки:

    Номер карты - номер бонусной карты клиента
    
    Сумма - сумма за заказ
    
    Чек-заказ - чек на покупку
    
    дата - дата покупки
    

In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [119]:
# Загрузка данных
df = pd.read_excel('Sales_ross.xlsx')


In [120]:
df

,Номер карты,Сумма,Чек-заказ,дата
0,43626.00,1238.00,184756,2022-12-07 15:12:50.1658034 +05:00
1,52678.00,90.00,375145,2022-12-07 15:09:47.7895676 +05:00
2,52678.00,94.00,375145,2022-12-07 15:09:47.7895676 +05:00
3,217604.00,94.00,184755,2022-12-07 15:05:32.9520864 +05:00
4,149098.00,138.00,375144,2022-12-07 15:00:10.2971950 +05:00
...,...,...,...,...
188935,97090.00,319.00,17,2021-12-29 15:31:49.2544233 +05:00
188936,97090.00,798.00,17,2021-12-29 15:31:49.2544233 +05:00
188937,97090.00,52.00,17,2021-12-29 15:31:49.2554233 +05:00
188938,97090.00,96.00,17,2021-12-29 15:31:49.2554233 +05:00


In [121]:
# Общая информация о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188940 entries, 0 to 188939
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Номер карты  168985 non-null  float64
 1   Сумма        188940 non-null  float64
 2   Чек-заказ    188940 non-null  int64  
 3   дата         188940 non-null  object 
dtypes: float64(2), int64(1), object(1)
memory usage: 5.8+ MB


In [122]:
# Перводим тип данных из колонки [дата] в формат даты
df['дата'] = pd.to_datetime(df['дата'])

In [123]:
# Извлекаем из данных только дату
df['дата'] = df['дата'].dt.date

In [124]:
df['дата'] = pd.to_datetime(df['дата'])

In [125]:
# Изменение названия колонок
df.rename(columns={'Чек-заказ': 'invoiceNo',
                   'Номер карты': 'customer',
                   'дата': 'date',
                   'Сумма': 'amount'}, inplace=True)

In [126]:
df

,customer,amount,invoiceNo,date
0,43626.00,1238.00,184756,2022-12-07
1,52678.00,90.00,375145,2022-12-07
2,52678.00,94.00,375145,2022-12-07
3,217604.00,94.00,184755,2022-12-07
4,149098.00,138.00,375144,2022-12-07
...,...,...,...,...
188935,97090.00,319.00,17,2021-12-29
188936,97090.00,798.00,17,2021-12-29
188937,97090.00,52.00,17,2021-12-29
188938,97090.00,96.00,17,2021-12-29


In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188940 entries, 0 to 188939
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   customer   168985 non-null  float64       
 1   amount     188940 non-null  float64       
 2   invoiceNo  188940 non-null  int64         
 3   date       188940 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 5.8 MB


In [129]:
# Проверка пропущенных значений в столбце с картами клиентов
df.customer.isna().sum()

19955

19955 клиентов при покупке не имели бонусной карты, информации(контактных данных) по этим клиентам никакой нет, можно исключить их из анализа, так как по результатам анализа запланирована рассылка, настроенная на узкие сегменты 

In [132]:
# Удаление строк с пропущенными значениями в колонке customer
df.dropna(subset='customer', inplace=True)

In [133]:
df['customer'] = df['customer'].astype(int)

In [134]:
# Определение последней даты
last_date = df.date.max()

In [135]:
last_date

Timestamp('2022-12-07 00:00:00')

In [136]:
# Построение RFM - таблицы
# группировка по картам клиентов, выцчисление давности посещения,частоту покупок, общая сумма покупок клиента  
rfm_table = df.groupby('customer', as_index=False) \
              .agg({'date': lambda x: (last_date - x.max()).days,  # Recency
                    'invoiceNo': 'count',  # Frequency
                    'amount': 'sum'})  # Monetary value

rfm_table['date'] = rfm_table['date'].astype(int)

rfm_table.rename(columns={'date': 'recency',
                          'invoiceNo': 'frequency',
                          'amount': 'monetary'}, 
                 inplace=True)

rfm_table


,customer,recency,frequency,monetary
0,10,177,22,3586.96
1,14,298,8,836.40
2,30,149,6,2294.00
3,48,121,2,276.00
4,60,25,128,61588.02
...,...,...,...,...
12243,294982,1,5,756.02
12244,294996,0,1,0.02
12245,294998,0,2,138.02
12246,295004,0,4,1202.58


In [137]:
# Определеине границ (квантили)
quantiles = rfm_table.quantile(q=[0.33, 0.66])

quantiles

,customer,recency,frequency,monetary
0.33,65407.14,44.00,3.00,918.00
0.66,207016.32,147.00,10.00,3276.04


Всего получается 3 группы клиентов, разделенные по квартилям по 33%

In [138]:
# удаление столбца customer (карты клиентов)
rfm_table.drop('customer', axis=1)

,recency,frequency,monetary
0,177,22,3586.96
1,298,8,836.40
2,149,6,2294.00
3,121,2,276.00
4,25,128,61588.02
...,...,...,...
12243,1,5,756.02
12244,0,1,0.02
12245,0,2,138.02
12246,0,4,1202.58


In [139]:
# присвоение нового имени для таблицы rfm_table
rfm_segmentation = rfm_table

In [147]:
# Классификация для групп по частоте покупок и общей сумме покупок клиента
def FMClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.33]:
        return 1
    elif value <= quantiles_table[parameter_name][0.66]:
        return 2
    else:
        return 3

# Классификация для групп по давности
def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.33]:
        return 3
    elif value <= quantiles_table[parameter_name][0.66]:
        return 2
    
    else:
        return 1

In [141]:
# Сегментация
rfm_segmentation['R_Quartile'] = rfm_segmentation['recency'].apply(RClass, args=('recency',quantiles))
rfm_segmentation['F_Quartile'] = rfm_segmentation['frequency'].apply(FMClass, args=('frequency',quantiles))
rfm_segmentation['M_Quartile'] = rfm_segmentation['monetary'].apply(FMClass, args=('monetary',quantiles))

# приведение значения в каждой колонке к строковому типу, что бы соединить в один RFM сегмент
rfm_segmentation['RFMClass'] = rfm_segmentation.R_Quartile.map(str) \
                              + rfm_segmentation.F_Quartile.map(str) \
                              + rfm_segmentation.M_Quartile.map(str)


In [142]:
# Таблица с итоговыми сегментами (111-333, где 111 - это клиенты редкие с 1-2 покупками, 333 - ядро, самые лояльные клиенты)
rfm_segmentation

,customer,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile,RFMClass
0,10,177,22,3586.96,1,3,3,133
1,14,298,8,836.40,1,2,1,121
2,30,149,6,2294.00,1,2,2,122
3,48,121,2,276.00,2,1,1,211
4,60,25,128,61588.02,3,3,3,333
...,...,...,...,...,...,...,...,...
12243,294982,1,5,756.02,3,2,1,321
12244,294996,0,1,0.02,3,1,1,311
12245,294998,0,2,138.02,3,1,1,311
12246,295004,0,4,1202.58,3,2,2,322


In [143]:
pd.set_option("display.float_format", "{:.2f}".format)

In [149]:
#LTV каждой из подгрупп
RFMClass = rfm_segmentation.groupby('RFMClass').agg({'monetary':'sum'})
RFMClass

,monetary
RFMClass,
111,622527.96
112,875367.96
113,109224.78
121,313566.02
122,1535814.68
123,1016311.00
131,2707.56
132,315333.10
133,1839792.94


In [150]:
# Вычисление доли суммы покупок каждой подгруппы в процентах 
(RFMClass.monetary / RFMClass.monetary.sum()) * 100

RFMClass
111    1.13
112    1.58
113    0.20
121    0.57
122    2.78
123    1.84
131    0.00
132    0.57
133    3.33
211    0.58
212    0.92
213    0.13
221    0.39
222    2.86
223    2.57
232    1.05
233   16.03
311    0.24
312    0.49
313    0.07
321    0.20
322    2.07
323    2.38
331    0.00
332    1.25
333   56.78
Name: monetary, dtype: float64

Общая сумма покупок Ядра лояльных клиентов, сегмент 333, составил 56,8 % 

In [153]:
# Лучшие клиенты (сегмент 333)
best = rfm_segmentation[rfm_segmentation['RFMClass'] == '333']
best

,customer,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile,RFMClass
4,60,25,128,61588.02,3,3,3,333
10,686,20,38,12799.02,3,3,3,333
22,2144,34,24,7773.12,3,3,3,333
29,3358,38,59,24393.20,3,3,3,333
58,6016,9,34,7020.02,3,3,3,333
...,...,...,...,...,...,...,...,...
12060,290026,1,25,9093.02,3,3,3,333
12191,293878,1,90,27869.30,3,3,3,333
12209,294288,2,57,16737.72,3,3,3,333
12220,294494,1,56,9987.02,3,3,3,333


In [155]:
best.customer.count() / rfm_segmentation.customer.count() * 100

17.823318092749837

Доля лучших клиентов составляет 17,8 %

In [146]:
# Редкие клиенты, сделавшие 1-4 покупки  (сегмент 111)
rfm_segmentation[rfm_segmentation['RFMClass'] == '111']

,customer,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile,RFMClass
7,274,229,1,198.00,1,1,1,111
9,384,315,2,218.00,1,1,1,111
14,1078,164,2,620.00,1,1,1,111
28,3252,212,2,676.00,1,1,1,111
40,4154,158,1,918.00,1,1,1,111
...,...,...,...,...,...,...,...,...
10893,264860,151,3,240.02,1,1,1,111
10904,265028,149,1,170.00,1,1,1,111
10905,265030,149,2,198.02,1,1,1,111
10909,265090,148,3,264.02,1,1,1,111


Ядро составляет 17,8 % от общего количества гостей и приносят 56,8 % дохода.

В перспективе есть возможность увеличить ядро за счет 133, 233, 313, 332 групп.

В более долгой перспективе доля спящих (группы 211-233) может стать ядром, при многоразовой стимуляции.

Группы 111-113 - скорее всего это ушедшие клиенты, не стоит тратить слишком много усилий,но можно запустить автоматическую цепочку реактивации.

Группы 121-223 - необходимо добиться обратной связи, выяснить причину ухода, расшевелить акциями, программами лояльности

311,312,313 - Welcome цепочка, показать преимущества, активно заинтересовать

323 - обычные рассылки для поддержания интереса

321, 322,331, 332 - можно увеличить сумму чека(специальные предложения, допродажи)

333 - отправлять особые предложения



Спасибо за внимание!